# Genetic Quantum Algorithm

In [ ]:
%pip install numpy matplotlib qiskit qiskit-aer tqdm

In [ ]:
# Import necessary libraries.

import copy
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit_aer.primitives import SamplerV2 as Sampler
from tqdm import tqdm

# Global parameters.

MAX_GENERATION = 100
MIN_ITEMS = 2
MAX_ITEMS = 30
STEPS = 1

In [ ]:
# Measure the quantum circuit.
def make(qc: QuantumCircuit) -> list:
    qc.measure_all()

    simulator = Sampler()
    job = simulator.run([qc], shots=1)
    result = job.result()
    counts = result[0].data.meas.get_counts()

    binary = list(counts.keys())[0]
    binary_list = [int(bit) for bit in binary]
    
    return binary_list

# Initializing the circuit with Hadamard gates for superposition.
def initialize(num_items: int) -> QuantumCircuit:
    qc= QuantumCircuit(num_items)
    for i in range(num_items):
        qc.h(i)
    
    return qc

# Repair the knapsack.
def repair(weights: list, capacity: float, binary: list, num_items: int) -> list:
    w = np.array(weights)
    x = np.array(binary)
    i, j = 0, 0

    overfilled = False
    if w @ x > capacity:
        overfilled = True

    while overfilled:
        i = np.random.randint(0, num_items)
        x[i] = 0
        if w @ x <= capacity:
            overfilled = False

    while not overfilled:
        j = np.random.randint(0, num_items)
        x[j] = 1
        if w @ x > capacity:
            overfilled = True

    x[j] = 0

    return x

# Compute profit.
def evaluate(values: list, binary: list) -> float:
    p = np.array(values)
    x = np.array(binary)

    return p @ x

# Get rotation angles.
def get_thetas(x: list, b: list, fx: float, fb: float, num_items: int) -> list:
    geq = fx >= fb

    thetas = []
    for i in range(num_items):
        if x[i] == 0 and b[i] == 0 and not geq:
            thetas.append(0.0)
        elif x[i] == 0 and b[i] == 0 and geq:
            thetas.append(0.0)
        elif x[i] == 0 and b[i] == 1 and not geq:
            thetas.append(0.0)
        elif x[i] == 0 and b[i] == 1 and geq:
            thetas.append(0.05 * np.pi)
        elif x[i] == 1 and b[i] == 0 and not geq:
            thetas.append(0.01 * np.pi)
        elif x[i] == 1 and b[i] == 0 and geq:
            thetas.append(0.025 * np.pi)
        elif x[i] == 1 and b[i] == 1 and not geq:
            thetas.append(0.005 * np.pi)
        elif x[i] == 1 and b[i] == 1 and geq:
            thetas.append(0.025 * np.pi)
        else:
            raise RuntimeError("There must a typo in one of these if statements.")

    return thetas

# Updating qubit's angle by thetas.
def update(qc: QuantumCircuit, thetas: list, num_items: int) -> QuantumCircuit:
    for i in range(num_items):
        qc.ry(thetas[i], i)

    return qc

# Perform genetic quantum algorithm.
def gqa(weights: list, values: list, capacity: float, num_items: int) -> tuple:
    t = 0

    qc_accumulated = initialize(num_items)
    qc_measured = copy.deepcopy(qc_accumulated)
    binary = make(qc_measured)
    
    best_solution = repair(weights, capacity, binary, num_items)
    best_profit = evaluate(values, best_solution)
    count_update = 1

    while t < MAX_GENERATION:
        t += 1

        qc_measured = copy.deepcopy(qc_accumulated)
        binary = make(qc_measured)

        solution = repair(weights, capacity, binary, num_items)
        profit = evaluate(values, solution)
        thetas = get_thetas(solution, best_solution, profit, best_profit, num_items)
        qc_accumulated = update(qc_accumulated, thetas, num_items)

        if profit > best_profit:
            best_profit = profit
            best_solution = solution
            count_update += 1

    return (best_profit, best_solution, count_update)

In [ ]:
# Collect data.

items = list(range(MIN_ITEMS, MAX_ITEMS+1, STEPS))
updates = []

for num_items in tqdm(range(MIN_ITEMS, MAX_ITEMS+1, STEPS), desc="Genetic Quantum Algorithm (Simulated)"):
    weights = np.random.randint(1, 100, num_items)
    values = np.random.randint(1, 100, num_items)
    capacity = 0.5 * sum(weights)

    (best_profit, best_solution, count_update) = gqa(weights, values, capacity, num_items)
    updates.append(count_update)

# Plot data.

plt.plot(items, updates, color='blue', label='Number of Updates')
plt.legend()

plt.xlabel("Number of Items")
plt.ylabel("Number of Updates")
plt.title("Updates on Genetic Quantum Algorithm (Simulated)")

plt.show()

# Print data.

print(f"Items: {items}")
print(f"Updates: {updates}")